In [99]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

ds = load_dataset("rajpurkar/squad")

model = AutoModelForCausalLM.from_pretrained("./cqatrains6/")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
model.eval()

model.eval()
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
data=""
tokenized=tokenizer(data, padding='longest', max_length=512, truncation=True, return_tensors="pt")
tokenized

{'input_ids': tensor([[128000,     34,     25,  12174,     11,   1071,  39431,  14643,     11,
           1053,    539,    433,    387,   6555,    422,    584,   1436,      0,
           8442,    279,   2523,  85880,   3245,    374,   1268,   1364,   2751,
           1555,    279,   7147,     13,    358,    656,    539,   1518,   1268,
           1694,  33173,   1053,   1520,   1077,    311,    656,    430,     13,
           8170,    838,     11,    279,  74665,     11,  14760,    304,   2345,
          12174,     11,    499,   2466,  18754,      0,   1364,   3287,   1431,
            733,   1555,    433,     26,   1364,   1193,   3463,   1364,   1550,
             13,   8489,     11,   1243,     11,   1071,  23720,  14295,     11,
            358,   1390,    311,   1781,    433,   2288,     13,   8155,   3814,
            994,    358,    574,    304,   4950,    358,   6818,    311,    733,
            311,   6212,     11,    323,    311,    636,   1555,    279,   7147,
             2

In [183]:
output = model(input_ids=tokenized['input_ids'], attention_mask=tokenized['attention_mask'], labels=tokenized['input_ids'])

In [184]:
logits=output.logits

In [185]:
import torch
import torch.nn.functional as F

probabilities = torch.nn.functional.softmax(output.logits, dim=-1)
most_likely_token = torch.argmax(probabilities[:, :, :], dim=-1)
print(most_likely_token)
most_likely_token = tokenizer.decode(most_likely_token[0])
print(most_likely_token)

tensor([[29749,    34,    34,   763,   931,  1071, 39431, 14643,   660,   387,
           539,   433,   387,  6555,    32,  1390,  1436,    32,  5995, 10973,
            32, 14643,  3201,   502,  1790,   574,  2751,   387,   279,   315,
           679,  1541,   358,   433,   433,   387,   279,   574,   539,  1520,
          7102,  1520,   539,  7077,   539,   433,   596,  5131,  7147,   422,
           596,   994,   220,   220,    11,  1071,   358,  1077,  3254,    32,
          2751,  1555,  3320,  1555,   279,   374,    26,  3287,   706,   922,
          3287,   539,  1077, 17872,   596,   279,   279, 39431,   358,   323,
           679,   656,   539,   656,  1781,    26,   574,  8489,  8155,  3814,
           279,   656,   264,   279,  4950,   358,   264,  6212,  3201,   387,
          6212,    26,  6212,  6212,   369,   433,  7147,    13,  3005,  1077,
           358,   574,   539,  6212,   358, 15447,  2574,   682,  2288,  2288,
           279,  1561,   323,  1541,   264,    26,  